In [85]:
import pandas as pd
import sqlite3
import math


def init_db(csv_file):
    df = pd.read_csv(csv_file)
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')  
    conn = sqlite3.connect('covid_data.db')
    df.to_sql('covid', conn, if_exists='replace', index=False)
    return conn

def execute_query(conn, query):
    result = pd.read_sql(query, conn)
    return result

def calculate_stdev(data):
    variance = data['VarianceRecovered']
    stdev = math.sqrt(variance)
    return stdev


csv_file = 'corona.csv'
conn = init_db(csv_file)




In [80]:
# 1. Write a code to check NULL values
query1= """
SELECT *
FROM covid
WHERE Province IS NULL OR
      Country IS NULL OR
      Latitude IS NULL OR
      Longitude IS NULL OR
      Date IS NULL OR
      Confirmed IS NULL OR
      Deaths IS NULL OR
      Recovered IS NULL;
"""
execute_query(conn, query1)

,Province,Country,Latitude,Longitude,Date,Confirmed,Deaths,Recovered


In [72]:
# 2.If NULL values are present, update them with zeros for all columns. 

query2 = """
UPDATE covid
SET 
    Province = COALESCE(Province, ''),
    Country = COALESCE(Country, ''),
    Latitude = COALESCE(Latitude, 0),
    Longitude = COALESCE(Longitude, 0),
    Date = COALESCE(Date, '0000-00-00'),  -- Assuming Date is in YYYY-MM-DD format
    Confirmed = COALESCE(Confirmed, 0),
    Deaths = COALESCE(Deaths, 0),
    Recovered = COALESCE(Recovered, 0);

"""
execute_query(conn, query2)

No NULL values found in the covid table.


In [81]:
# 3. check total number of rows
query3= """
SELECT COUNT(*) as TotalRows
FROM covid;

"""
execute_query(conn, query3)

,TotalRows
0,78386


In [61]:
# 4.Check what is start_date and end_date
query4 = """
SELECT MIN(Date) as StartDate,
       MAX(Date) as EndDate
FROM covid;
"""
execute_query(conn, query4)

,StartDate,EndDate
0,2020-01-22 00:00:00,2021-06-13 00:00:00


In [60]:
# 5. Number of month present in dataset

query5 = """
SELECT COUNT(DISTINCT strftime('%m-%Y', Date)) as NumberOfMonths
FROM covid;

"""
execute_query(conn, query5)

,NumberOfMonths
0,18


In [66]:
# 6. Find monthly average for confirmed, deaths, recovered

query6 = """
SELECT 
    strftime('%m-%Y', Date) as Month,
    AVG(Confirmed) as AverageConfirmed,
    AVG(Deaths) as AverageDeaths,
    AVG(Recovered) as AverageRecovered
FROM covid
GROUP BY Month;

"""
execute_query(conn, query6)

,Month,AverageConfirmed,AverageDeaths,AverageRecovered
0,01-2020,4.145455,0.123377,0.092857
1,01-2021,3911.228530,84.183703,1919.636992
2,02-2020,15.296014,0.593596,7.032020
3,02-2021,2433.363636,69.164889,1558.391698
4,03-2020,161.130289,8.660662,27.873900
5,03-2021,2916.797235,59.199832,1652.285924
6,04-2020,505.800433,41.522294,171.642208
7,04-2021,4699.355195,78.438745,3074.785065
8,05-2020,574.849811,30.280897,318.296397
9,05-2021,4005.254085,76.780268,4007.507750


In [65]:
# 7 .Find most frequent value for confirmed, deaths, recovered each month 
query7= """
WITH MonthlyCounts AS (
    SELECT 
        strftime('%m-%Y', Date) as Month,
        Confirmed,
        Deaths,
        Recovered,
        COUNT(*) as Frequency
    FROM covid
    GROUP BY Month, Confirmed, Deaths, Recovered
    ORDER BY Frequency DESC
)
SELECT 
    Month,
    MAX(Confirmed) as MostFrequentConfirmed,
    MAX(Deaths) as MostFrequentDeaths,
    MAX(Recovered) as MostFrequentRecovered
FROM MonthlyCounts
GROUP BY Month;

"""
execute_query(conn, query7)

,Month,MostFrequentConfirmed,MostFrequentDeaths,MostFrequentRecovered
0,01-2020,2131,49,51
1,01-2021,300462,4475,87090
2,02-2020,14840,242,3418
3,02-2021,134975,3907,98389
4,03-2020,26314,1085,4289
5,03-2021,100158,3869,102138
6,04-2020,50740,2607,33227
7,04-2021,401993,4249,299988
8,05-2020,34907,2309,51717
9,05-2021,414188,4529,422436


In [64]:
# 8. Find minimum values for confirmed, deaths, recovered per year
query8 = """
SELECT 
    strftime('%Y', Date) as Year,
    MIN(Confirmed) as MinConfirmed,
    MIN(Deaths) as MinDeaths,
    MIN(Recovered) as MinRecovered
FROM covid
GROUP BY Year;

"""
execute_query(conn, query8)

,Year,MinConfirmed,MinDeaths,MinRecovered
0,2020,0,0,0
1,2021,0,0,0


In [63]:
# 9. Find maximum values of confirmed, deaths, recovered per year
query9 = """
SELECT 
    strftime('%Y', Date) as Year,
    MAX(Confirmed) as MaxConfirmed,
    MAX(Deaths) as MaxDeaths,
    MAX(Recovered) as MaxRecovered
FROM covid
GROUP BY Year;

"""
execute_query(conn, query9)

,Year,MaxConfirmed,MaxDeaths,MaxRecovered
0,2020,823225,3752,1123456
1,2021,414188,7374,422436


In [62]:
# 10. The total number of case of confirmed, deaths, recovered each month
query10 = """
SELECT 
    strftime('%m-%Y', Date) as Month,
    SUM(Confirmed) as TotalConfirmed,
    SUM(Deaths) as TotalDeaths,
    SUM(Recovered) as TotalRecovered
FROM covid
GROUP BY Month;

"""
execute_query(conn, query10)

,Month,TotalConfirmed,TotalDeaths,TotalRecovered
0,01-2020,6384,190,143
1,01-2021,18672205,401893,9164347
2,02-2020,68312,2651,31405
3,02-2021,10492664,298239,6719785
4,03-2020,769236,41346,133070
5,03-2021,13924790,282620,7888013
6,04-2020,2336798,191833,792987
7,04-2021,21711021,362387,14205507
8,05-2020,2744333,144561,1519547
9,05-2021,19121083,366549,19131842


In [45]:
# 11. Check how coronavirus spread out with respect to confirmed case
query11 ="""
SELECT 
SUM(Confirmed) as TotalConfirmed,
AVG(Confirmed) as AverageConfirmed,
AVG(Confirmed * Confirmed) - AVG(Confirmed) * AVG(Confirmed) as VarianceConfirmed
FROM covid;
"""
execute_query(conn, query11)

,TotalConfirmed,AverageConfirmed,VarianceConfirmed
0,169065144,2156.828311,1.572889e+08


In [49]:
# 12.Check how corona virus spread out with respect to death case per month
query12 = """
SELECT
strftime('%m-%Y', Date) as Month,
SUM(Deaths) as TotalDeaths,
AVG(Deaths) as AverageDeaths,
AVG(Deaths * Deaths) - AVG(Deaths) * AVG(Deaths) as VarianceDeaths
FROM covid
GROUP BY Month;

"""
execute_query(conn, query12)

,Month,TotalDeaths,AverageDeaths,VarianceDeaths
0,01-2020,190,0.123377,4.245817
1,01-2021,401893,84.183703,102758.432319
2,02-2020,2651,0.593596,68.321849
3,02-2021,298239,69.164889,68478.871467
4,03-2020,41346,8.660662,3900.792265
5,03-2021,282620,59.199832,54385.969703
6,04-2020,191833,41.522294,40504.268118
7,04-2021,362387,78.438745,94611.470923
8,05-2020,144561,30.280897,20684.911671
9,05-2021,366549,76.780268,131769.469313


In [52]:
# 13. Check how corona virus spread out with respect to recovered caseSELECT 
query13 = """
SELECT 
    strftime('%m-%Y', Date) as Month,
    SUM(Recovered) as Total_Recovered,
    AVG(Recovered) as Average_Recovered,
    AVG(Recovered * Recovered) - AVG(Recovered) * AVG(Recovered) as Variance_Recovered
FROM covid
GROUP BY Month;

"""
execute_query(conn, query13)

,Month,Total_Recovered,Average_Recovered,Variance_Recovered
0,01-2020,143,0.092857,2.633585e+00
1,01-2021,9164347,1919.636992,3.149370e+07
2,02-2020,31405,7.032020,1.244666e+04
3,02-2021,6719785,1558.391698,2.442741e+07
4,03-2020,133070,27.873900,4.011319e+04
5,03-2021,7888013,1652.285924,3.489739e+07
6,04-2020,792987,171.642208,7.698930e+05
7,04-2021,14205507,3074.785065,2.244196e+08
8,05-2020,1519547,318.296397,1.978206e+06
9,05-2021,19131842,4007.507750,7.551755e+08


In [53]:
# 14. Find Country having highest number of the Confirmed case
query14 = """
SELECT Country, MAX(Confirmed) as MaxConfirmed
FROM covid;
"""
execute_query(conn, query14)

,Country,MaxConfirmed
0,Turkey,823225


In [50]:
# 15. Find Country having lowest number of the death case

query15 = """ 
SELECT Country, SUM(Deaths) as TotalDeaths
FROM covid
GROUP BY Country
ORDER BY TotalDeaths ASC
LIMIT 1;
"""
execute_query(conn, query15)


,Country,TotalDeaths
0,Dominica,0


In [84]:
# 16 . Find top 5 countries having highest recovered case
query16 = """
SELECT Country, SUM(Recovered) as TotalRecovered
FROM covid
GROUP BY Country
ORDER BY TotalRecovered DESC
LIMIT 5;
"""
    

    
# Execute queries
execute_query(conn, query16)
    
    

,Country,TotalRecovered
0,India,28089649
1,Brazil,15400169
2,US,6303715
3,Turkey,5202251
4,Russia,4745756
